# STRIDE: Street View-based Environmental Feature Detection and Pedestrian Collision Prediction - Tutorial

<img src="images/STRIDE_benchmark.png"/>

Welcome to the **STRIDE** tutorial! In this tutorial, we will go through some primary usage of STRIDE, including the following:
* Sample random points over the streets of Bogotá and other Latin American cities.
* Download panoramic images from the Google Streetview API that correspond to the sampled points.
* Run the STRIDE model on the downloaded images
* Visualize the results of the model in built environment detection and pedestrian collision estimation

You can run this tutorial on your server either on GPUs or CPUs, but the latter will take significantly longer.

## 1) Install Dependencies

<img src=https://miro.medium.com/v2/resize:fit:786/format:webp/1*IMGOKBIN8qkOBt5CH55NSw.png/>

Your first task is to create and configure the conda environment with all the dependencies required to run the code. This includes:

- **PyTorch:** Deep Learning library to handle the entire development process of Deep Neural Networks
- **Torch Vision:** Computer Vision library with state-of-the-art computer vision architectures and functions
- **Cuda Toolkit:** Library to handle the GPU mounting process of PyTorch models
- **GeoPandas:** Data Handling library to process geographic and coordinates data.

In this step, we will also download all necessary files to run the model. These include:

- **city_coordinates.json** JSON file with information about city boundaries to sample points and plot them on maps.
- **STRIDE_weights.pth** PyTorch file with the values of all the model's pretrained parameters (weights).

In [ ]:
conda install pytorch==1.9.0 torchvision==0.10.0 cudatoolkit=11.1 -c pytorch -c nvidia

pip install -r requirements.txt

cd models/dino/ops
python setup.py build install
python test.py
cd ../../.. 

## 2) Variable setting

In this step, we will import son basic libraries and set the following base variables for the demo:

- ```city``` name of the city to sample points from
- ```num_images``` number of points and images to sample
- ```city_coordinates_file``` name of the JSON with the boundary coordinates of each city
- ```pretrained_model``` path to the PyTorch file with the model parameters (weights)

In [ ]:
import json
import random
import re
import warnings
warnings.filterwarnings("ignore")

# Test city to download images
city = "sao paulo"

# Number of images to download
num_images = 3

# File with limit coordinates per city
city_coordinates_file = "city_coordinates.json"

# Path to the pretrained model parameters (weights)
pretrained_model_file = "pretrained_models/stride_fold1.pth"

# Path to the categories file
categories_file = "categories.txt"

print('')

## 3) Sample Points

In this step, we will sample the selected number of random points in the selected city. For this purpose, we will sample random coordinates within the established boundaries of the city that will correspond to the sampled points. Then, we will plot the points on a map of the city.

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx
from pyproj import Transformer

#--------------- SAMPLE RANDOM POINTS 


# Load city boundary coordinates file
with open(city_coordinates_file, 'r') as f:
    min_max_coordinates = json.load(f)
    
# Get coordinates for the selected city
min_max_coordinates = min_max_coordinates[city]
bounds = min_max_coordinates['bounds']
seed = min_max_coordinates['seed']
random.seed(seed)

# Establish min max coordinate boundaries
min_lat = min_max_coordinates['latitude']['min']
max_lat = min_max_coordinates['latitude']['max']

min_lon = min_max_coordinates['longitude']['min']
max_lon = min_max_coordinates['longitude']['max']


# Generate random coordinates inside the boundaries
coordinates = []
geometry = []
for i in range(num_images):
    # Random nfloat generator within boundaries
    rand_lat = random.uniform(min_lat, max_lat)
    rand_lon = random.uniform(min_lon, max_lon)
    coordinates.append((rand_lat, rand_lon))
    
    # Convert the list of coordinates into GeoDataFrame points
    geometry.append(Point((rand_lon, rand_lat)))
    print((rand_lat, rand_lon))

#--------------- PLOT POINTS ON MAP

# Create a GeoDataFrame from the sampled points
gdf_points = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")

# Convert the coordinates to the web mercator projection (EPSG:3857) for contextily
gdf_points = gdf_points.to_crs(epsg=3857)

# Plot the points on a street map of the city
fig, ax = plt.subplots(figsize=(10, 10))
gdf_points.plot(ax=ax, color='red', markersize=100, alpha=0.6)

# Set the extent to the entire Bogotá area
ax.set_xlim([bounds['minx'], bounds['maxx']]) 
ax.set_ylim([bounds['miny'], bounds['maxy']]) 
# ax.set_xlim([-5210000, -5170000]) 
# ax.set_ylim([-2730000, -2680000]) 

# Add a basemap from contextily (OpenStreetMap or similar)
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Convert back to latitude and longitude for correct axis labels
# Create a transformer to go from Web Mercator (EPSG:3857) to WGS 84 (EPSG:4326)
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

# Transform x and y axis limits back to lat/lon
min_lon, min_lat = transformer.transform(bounds['minx'], bounds['miny'])
max_lon, max_lat = transformer.transform(bounds['maxx'], bounds['maxy'])

# Set the ticks for latitude and longitude
ax.set_xticks([bounds['minx'], (bounds['minx'] + bounds['maxx']) / 2, bounds['maxx']])
ax.set_xticklabels([f"{min_lon:.2f}°", f"{(min_lon + max_lon) / 2:.2f}°", f"{max_lon:.2f}°"])

ax.set_yticks([bounds['miny'], (bounds['miny'] + bounds['maxy']) / 2, bounds['maxy']])
ax.set_yticklabels([f"{min_lat:.2f}°", f"{(min_lat + max_lat) / 2:.2f}°", f"{max_lat:.2f}°"])

# Add titles and labels
plt.title(f'Coordinates in {city}')

# Show the plot
plt.show()

## 4) Get Metadata of Panoramic Image Candidates

<img src="https://lh3.googleusercontent.com/beHNDtluUwK6HO2MAcsEjo_K5fgAOR4G15SzFYkAE9nVc66hgeZw0of5YtVW1X-HHx6PZHgXlUlMFA2Ss0VxsST3jUOWnafszFXgpVO4gs92bnmylQ"/>

In this step, we will get the data for the best panoramic image for each point. For each point, we will follow the following order: 

1. Send a request to the Google Streetview API to get a list of possible panoramic images that are close to the point.
2. Reformat the API response into dictionaries with the Panorama ID, latitude, longitude, and date of the candidates.
3. Find the candidate closest to the sampled point using the Haversine distance.

This part of the demo has been adapted from the [streetview](https://github.com/robolyst/streetview) library.

In [ ]:
import requests
import numpy as np
from haversine import haversine, Unit

# List of panorama
coordinate_panos = []

for lat,lon in coordinates:
    # URL for the GET request
    url = (
            "https://maps.googleapis.com/maps/api/js/" # Google API url
            "GeoPhotoService.SingleImageSearch" # Image search engine
            "?pb=!1m5!1sapiv3!5sUS!11m2!1m1!1b0!2m4!1m2!3d{0:}!4d{1:}!2d50!3m10" # searching code
            "!2m2!1sen!2sGB!9m1!1e2!11m4!1m3!1e2!2b1!3e2!4m10!1e1!1e2!1e3!1e4"
            "!1e8!1e6!5m1!1e2!6m1!1e2"
            "&callback=callbackfunc"
        )
    
    # Insert latitud and longitud into the request url
    url = url.format(lat, lon)
    print(url)
    
    # GET request to the Google API to search for close images
    resp = requests.get(url).text
    
    # Re format the response into a dictionary
    blob = re.findall(r"callbackfunc\( (.*) \)$", resp)[0]
    data = json.loads(blob)

    # Handle empty responses
    if data == [[5, "generic", "Search returned no images."]]:
        panos = []

    subset = data[1][5][0]

    raw_panos = subset[3][0]

    if len(subset) < 9 or subset[8] is None:
        raw_dates = []
    else:
        raw_dates = subset[8]

    # For some reason, dates do not include a date for each panorama.
    # the n dates match the last n panos. Here we flip the arrays
    # so that the 0th pano aligns with the 0th date.
    raw_panos = raw_panos[::-1]
    raw_dates = raw_dates[::-1]

    dates = [f"{d[1][0]}-{d[1][1]:02d}" for d in raw_dates]
    

    first_panorama_data = None
    min_haversine = float('inf')
    
    # Iterate over all image candidates and reformat into dictionary
    print(f'Available Panoramas for coordinates {(lat,lon)}:')
    for i, pano in enumerate(raw_panos):
        this_panorama = {
                        "pano_id": pano[0][1],
                        "latitude": pano[2][0][2],
                        "longitude": pano[2][0][3],
                        "date": dates[i] if i < len(dates) else None,
                        # # One could also retrive data about image capturing like heading, pitch, roll, etc.
                        # "heading":pano[2][2][0],
                        # "pitch":pano[2][2][1] if len(pano[2][2]) >= 2 else None,
                        # "roll":pano[2][2][2] if len(pano[2][2]) >= 3 else None,
                        # "date":dates[i] if i < len(dates) else None,
                        # "elevation":pano[3][0] if len(pano) >= 4 else None,
                        } 
        print(i+1,this_panorama)

        # Check if it the candidate is the closest one
        this_distance = haversine((lat,lon), (this_panorama['latitude'], this_panorama['longitude']), unit=Unit.METERS)
        if this_distance<min_haversine:
            first_panorama_data = this_panorama
            min_haversine=this_distance
    
    coordinate_panos.append(first_panorama_data)
    print('Closes Panorama is:')
    print(first_panorama_data)
    print(f"Distance to point:\t{min_haversine:.0f}m\n")
    

## 5) Download Panoramic Images

<img src="https://kstatic.googleusercontent.com/files/0df80ab0ad7cc9a3d4a19b068da05a98f4a17ec4182738214b1bd8ac8263ab4a965de25694a9e4220c627ec3016bfce7b5cd3aa0b6afc1eb5a24d25ed2a3a788"/>

In this step, we will download the panoramic images whose metadata we selected in the previous step. Remember each image corresponds to a sampled point. For each image/point, we will follow the following order:

1. Establish image size according to the desired zoom of the image
2. Iteratively send requests to the Google Streetview API for each part of the image (parts or tiles must request images) and stitch them all together in the process
3. Preprocess the image to remove empty zones caused by the download process.
4. Preprocess the image to remove a fraction of its top and bottom to remove image artifacts caused by panoramic image creation.
5. Plot the image to visualize it.
6. Keep the exact coordinates of the image, as they might be slightly far from the point, and the model requires the coordinates of the input image.

This part of the demo has also been adapted from the [streetview](https://github.com/robolyst/streetview) library.

In [ ]:
from io import BytesIO
import itertools
from PIL import Image
from tqdm import tqdm
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 400

# Declare lists of images and image coordinates
panoramas = []
pano_coordinates = []

# Declare constant variables for image gathering, formation and preprocessing
zoom = 5
crop_height = 4000
typical_height = 6656
typical_width = 13312
crop_index = 1000

# Iterate over selected panorama metadata
for main_panorama in coordinate_panos:
    tile_width = 512
    tile_height = 512
    
    # Define image size according to zoom
    width, height = 2**zoom, 2 ** (zoom - 1)

    # Create empty image variable to be filled with the downloaded tiles
    panorama = Image.new("RGB", (width * tile_width, height * tile_height))
    
    # Itearte over the x,y coordinates of each tile of the image to download
    for x, y in tqdm(itertools.product(range(width), range(height)), total=width*height, desc=f"Downloading and stitching {main_panorama['pano_id']}"):
        
        # URL for the GET request of the corresponding tile
        tile_url = f"https://cbk0.google.com/cbk?output=tile&panoid={main_panorama['pano_id']}&zoom={zoom}&x={x}&y={y}"
        
        # Send GET request to the Google Streetview API
        response = requests.get(tile_url, stream=True)

        # Reformat response as an image corresponding to the tile
        tile = Image.open(BytesIO(response.content))

        # Paste the tile on the image
        panorama.paste(im=tile, box=(x * tile_width, y * tile_height))

        # Delete tile to save space
        del tile
    
    # # Remove empty parts of the image left after the filling process
    # pano_width, pano_height = panorama.size
    # if pano_width>typical_width:
    #     panorama = panorama.crop(box=(0, 0, typical_width, pano_height))
    #     pano_width, pano_height = panorama.size
    # if pano_height>typical_height:
    #     panorama = panorama.crop(box=(0, 0, pano_width, typical_height))
    #     pano_width, pano_height = panorama.size
    
    # # Crop 1000 pixels from the top of the image and 1656 from the bottom to remove artifacts from panoramic omage formation
    # if pano_height>crop_height:
    #     panorama = panorama.crop(box=(0, crop_index, pano_width, crop_height + crop_index))
    
    # Plot image
    plt.imshow(np.array(panorama))
    plt.axis('off')
    plt.show()
    plt.close()

    # Append image and coordinates to lists
    panoramas.append(panorama) 
    pano_coordinates.append((main_panorama['latitude'], main_panorama['longitude']))   

## 6) Import and Build the Model

<img src="https://github.com/BCV-Uniandes/STRIDE/blob/main/images/STRIDE_baseline.png?raw=true"/>
In this step, we will build the model using the predefined classes in the STRIDE repo. First, we load a configuration file containing each hyperparameter's values regarding the model's design and architecture. Then, we will use these configurations to build the model using the following four modules:

1. ```Backbones``` This module builds the CNN backbone that initially processes the image to extract visual features
2. ```PositionEmbeddingSineHW``` This module uses a sine function to include information about the 2D position of the extracted visual features
3. ```DeformableTransformer``` This module builds the Transformer that processes and compares the visual features with the added positional encoding and the object queries
4. ```DINO``` This is the general module of the entire model that handles the model workflow, puts together all the parts, processes all outputs and inputs of each part, and generates final boxes and their classes.
5. ```postprocess``` This module cleans the prox predictions by removing empty boxes and scaling them to the original size

As you can see, all of these modules have multiple arguments that correspond to the model design's hyperparameters. The values of all these arguments and parameters are coded in the configuration file.

Our model is entirely based on the [DINO](https://github.com/IDEA-Research/DINO) model and code. 

In [ ]:
import torch
from util.slconfig import SLConfig
from models.dino.dino import DINO, PostProcess
from models.dino.deformable_transformer import DeformableTransformer
from models.dino.backbone import Backbone, FrozenBatchNorm2d, Joiner
from models.dino.position_encoding import PositionEmbeddingSineHW

# Hardcodded path to the configuration script
config_file = "./config/STRIDE/STRIDE_4scale.py"

# Parse configuration variables into atributes of a Config object
cfg = SLConfig.fromfile(config_file)

# Configura and build the Sine positional embedding
N_steps = cfg.hidden_dim // 2
position_embedding = PositionEmbeddingSineHW(N_steps, 
                        temperatureH=cfg.pe_temperatureH,
                        temperatureW=cfg.pe_temperatureW,
                        normalize=True
                    )

# Configura and build the CNN backbone
backbone = Backbone(cfg.backbone, cfg.lr_backbone > 0, cfg.dilation,   
                    cfg.return_interm_indices,   
                    batch_norm=FrozenBatchNorm2d,
                    regression=cfg.regression and cfg.backbone_reg,
                    out_feats=cfg.regression and cfg.backbone_out,
            )

# Join the CNN backbone with the Sine embedding
bb_num_channels = backbone.num_channels
backbone = Joiner(backbone, position_embedding)
backbone.num_channels = bb_num_channels 

# Configura and build the Detection Transformer
transformer = DeformableTransformer(
        d_model=cfg.hidden_dim,
        dropout=cfg.dropout,
        nhead=cfg.nheads,
        num_queries=cfg.num_queries,
        dim_feedforward=cfg.dim_feedforward,
        num_encoder_layers=cfg.enc_layers,
        num_unicoder_layers=cfg.unic_layers,
        num_decoder_layers=cfg.dec_layers,
        normalize_before=cfg.pre_norm,
        return_intermediate_dec=True,
        query_dim=cfg.query_dim,
        activation=cfg.transformer_activation,
        num_patterns=cfg.num_patterns,
        modulate_hw_attn=True,

        deformable_encoder=True,
        deformable_decoder=True,
        num_feature_levels=cfg.num_feature_levels,
        enc_n_points=cfg.enc_n_points,
        dec_n_points=cfg.dec_n_points,
        use_deformable_box_attn=cfg.use_deformable_box_attn,
        box_attn_type=cfg.box_attn_type,

        learnable_tgt_init=True,
        decoder_query_perturber=None,

        add_channel_attention=cfg.add_channel_attention,
        add_pos_value=cfg.add_pos_value,
        random_refpoints_xy=cfg.random_refpoints_xy,

        # two stage
        two_stage_type=cfg.two_stage_type,  # ['no', 'standard', 'early']
        two_stage_pat_embed=cfg.two_stage_pat_embed,
        two_stage_add_query_num=cfg.two_stage_add_query_num,
        two_stage_learn_wh=cfg.two_stage_learn_wh,
        two_stage_keep_all_tokens=cfg.two_stage_keep_all_tokens,
        dec_layer_number=cfg.dec_layer_number,
        rm_self_attn_layers=None,
        key_aware_type=None,
        layer_share_type=None,

        rm_detach=None,
        decoder_sa_type=cfg.decoder_sa_type,
        module_seq=cfg.decoder_module_seq,

        embed_init_tgt=cfg.embed_init_tgt,
        use_detached_boxes_dec_out=cfg.use_detached_boxes_dec_out
    )

# Configura and build the STRIDE model (based on the DINO model) that puts all parts together
model = DINO(
        backbone,
        transformer,
        num_classes=cfg.num_classes,
        num_queries=cfg.num_queries,
        aux_loss=True,
        iter_update=True,
        query_dim=4,
        random_refpoints_xy=cfg.random_refpoints_xy,
        fix_refpoints_hw=cfg.fix_refpoints_hw,
        num_feature_levels=cfg.num_feature_levels,
        nheads=cfg.nheads,
        dec_pred_class_embed_share=cfg.dec_pred_class_embed_share,
        dec_pred_bbox_embed_share=cfg.dec_pred_bbox_embed_share,
        # two stage
        two_stage_type=cfg.two_stage_type,
        # box_share
        two_stage_bbox_embed_share=cfg.two_stage_bbox_embed_share,
        two_stage_class_embed_share=cfg.two_stage_class_embed_share,
        decoder_sa_type=cfg.decoder_sa_type,
        num_patterns=cfg.num_patterns,
        dn_number = cfg.dn_number if cfg.use_dn else 0,
        dn_box_noise_scale = cfg.dn_box_noise_scale,
        dn_label_noise_ratio = cfg.dn_label_noise_ratio,
        dn_labelbook_size = cfg.dn_labelbook_size,
        # Regression
        regression=cfg.regression,
        reg_encoder=cfg.reg_encoder,
        backbone_reg=cfg.backbone_reg,
        backbone_out=cfg.backbone_out,
        reg_cls_embed=cfg.reg_cls_token,
        just_backbone=cfg.just_backbone,
        norm_funct=cfg.norm_funct,
        denoise=cfg.denoise,
        no_loss_norm=cfg.no_loss_norm
    )

# Configura and build the postprocessor
postprocessor = PostProcess(num_select=cfg.num_select, nms_iou_threshold=cfg.nms_iou_threshold)

print('Model Loaded')

## 7) Load Model on GPU and Load Parameters

<img src="https://i.blogs.es/57ca96/cuda/450_1000.webp"/>

In this step, we will first load the model in our GPU (if available). When we build the model, this one is, by default, built on your computers' processors (the CPUs). In order to make the model run faster, we should store it in our GPUs.

Then, we will load the model's trained parameters. When we built the model, it was initialized with random parameters; in this step, we will load the PyTorch file with the trained parameters of the model, and then we will update the previously built model with the ones that were just loaded.

In [ ]:
# Device: cuda (GPU) if it is available or cpu otherwise
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Mount model in the GPU
model.to(device)

# Load PyTorch file with the model weights
checkpoint = torch.load(pretrained_model_file, map_location='cpu')['model']

# Load the model weights into the model
_load_output = model.load_state_dict(checkpoint, strict=False)
print(_load_output)

## 8) Run Model on Images and Visualize Predictions

In this final step, we will iteratively run the model on the images and visualize its outputs. This iterative approach will help us refine our model and improve its performance. To do so, we will follow the following order for each image:

1. Preprocess images by downscaling them to fit in the GPU memory, normalizing their values, and parsing them into tensors.
2. Input images and their coordinates into the models.
3. Postprocess outputs into scaled bounding boxes.
4. Filter low-score bounding boxes.
5. Draw bounding boxes on top of images to visualize output.

In [ ]:
import datasets.transforms as T
from util.misc import nested_tensor_from_tensor_list
import matplotlib.patches as patches
import matplotlib.cm as cm

# Load categories names
with open(categories_file, 'r') as f:
    categories = [l.strip() for l in f]

# Establish color map to draw boxes
cmap = cm.get_cmap('gist_rainbow', 27)

# Built image preprocessing (resizing and normalization) function
image_transform = T.Compose([
                    T.RandomResize([cfg.img_size]),
                    T.ToTensor(),
                    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])

# Turn down all backpropagation attributes
with torch.no_grad():
    # SEt the model on inference mode and turn off any random layers
    model.eval()

    # Iterate over images and their coordinates
    for panorama, pano_coords in tqdm(zip(panoramas, pano_coordinates), total=len(panoramas), desc="Running inference on images."):
        w, h = panorama.size

        # Preprocess image (resize and normalize)
        processed_panorama = image_transform(panorama, {})[0]

        # Parse images into nested tensorss
        processed_panorama = nested_tensor_from_tensor_list([processed_panorama]).to(device)
        
        # Run the model
        outputs = model(samples=processed_panorama, 
                        targets=[{'latitude': torch.tensor([pano_coords[0]]).to(device), 'longitude': torch.tensor([pano_coords[1]]).to(device)}])
        
        # Postprocess outputs
        orig_target_sizes = torch.as_tensor([[int(h), int(w)]]).to(device)
        results = postprocessor(outputs, orig_target_sizes)[0]
        
        # Extract all information about predictions
        collitions = results["regression"]
        boxes = results["boxes"]
        labels = results['labels'].tolist()
        scores = results['scores'].tolist()
        
        # Change bounding boxes format
        xmin, ymin, xmax, ymax = boxes.unbind(1)
        boxes = torch.stack((xmin, ymin, xmax - xmin, ymax - ymin), dim=1).tolist()        
        
        # Reformat results into a single list filteirng those with less than 0.25 score
        results = [
                        {
                            "category_id": label,
                            "bbox": box,
                            "score": score,
                        }
                        for box,label,score in zip(boxes, labels, scores)
                        if score >= 0.25
                    ]
        
        # Take top 300 predictions
        results.sort(key=lambda x: x['score'], reverse=True)
        results = results[:300]
        
        
        # Plot the image
        fig, ax = plt.subplots(1)
        ax.imshow(panorama)
        plt.axis('off')
        
        for box in results:
            category = box['category_id']
            bbox = box['bbox']
            score = box['score']
            
            # Unpack bounding box coordinates
            x_min, y_min, b_w, b_h = bbox
                        
            # Create a rectangle patch
            rect = patches.Rectangle((x_min, y_min), b_w, b_h, linewidth=0.5, edgecolor=cmap((category-1)/27), facecolor='none')
            
            # Add the rectangle to the plot
            ax.add_patch(rect)
            
            # Add the category and score label
            label = f'{categories[category-1]}: {score:.2f}'
            ax.text(x_min, y_min, label, color='black', fontsize=1, 
                    bbox=dict(facecolor='white', alpha=0.4, pad=0, edgecolor=cmap((category-1)/27)))
        
        # Display the result
        plt.show()
        plt.close()